In [26]:
import pandas as pd
import json
import glob

To do:
- Mihalcea's multimodal deception: https://web.eecs.umich.edu/~mihalcea/downloads.html#OpenDeception
  

## Open Deception

Downloaded from Rada Mihalcea's [website](https://web.eecs.umich.edu/~mihalcea/downloads.html), based on her open-domain deception paper.

In [13]:
input_file = 'data/OpenDeception/7Truth7LiesDataset.csv'
df = pd.read_csv(input_file, quotechar="'", escapechar="\\")
df.head()

,id,_gender,age,education,country,text,class
0,1_f_l_1,Female,26,Bachelors degree,Canada,There is a great deal of truth to the anti-vax...,lie
1,1_f_l_2,Female,26,Bachelors degree,Canada,Jenny mccarthy is a learned doctor who deserve...,lie
2,1_f_l_3,Female,26,Bachelors degree,Canada,Driving doesn't really require any practice.,lie
3,1_f_l_4,Female,26,Bachelors degree,Canada,Drinking and driving is a winning and safe com...,lie
4,1_f_l_5,Female,26,Bachelors degree,Canada,Good hygiene isn't really important or attract...,lie


In [24]:
data = {}
data['pos'] = list(df[df['class']=='lie']['text']) # positive denotes lie
data['neg'] = list(df[df['class']=='truth']['text']) # negative denotes truth

In [25]:
output_file = 'output/open_deception.json'
with open(output_file, 'w') as outfile:
    json.dump(data, outfile)

## Fake News

Downloaded from Rada Mihalcea's [website](https://web.eecs.umich.edu/~mihalcea/downloads.html), based on her fake news paper.

In [44]:
files = glob.glob('data/fakeNewsDatasets/fakeNewsDataset/**/*.txt')
pos, neg = [], []
for file in files:
    with open(file, 'r') as f:
        contents = f.read()
        if 'legit' in file:
            neg.append(contents)
        else:
            pos.append(contents)

In [47]:
data = {}
data['pos'] = pos
data['neg'] = neg

In [48]:
output_file = 'output/fake_news.json'
with open(output_file, 'w') as outfile:
    json.dump(data, outfile)

## Real Life Deception

Downloaded from Rada Mihalcea's [website](https://web.eecs.umich.edu/~mihalcea/downloads.html), based on her real life deception paper.

In [49]:
files = glob.glob('data/RealLifeDeception/Transcription/**/*.txt')
pos, neg = [], []
for file in files:
    with open(file, 'r') as f:
        contents = f.read()
        if 'truth' in file:
            neg.append(contents)
        else:
            pos.append(contents)

In [52]:
data = {}
data['pos'] = pos # lie
data['neg'] = neg # truth

In [53]:
output_file = 'output/real_life_deception.json'
with open(output_file, 'w') as outfile:
    json.dump(data, outfile)